# Spot finder

A notebook to find spots from a scanned map.  
First we give it the map and cell definitions file locations.  
Also defining some classes to look nicer later.

In [1]:
import json

mapfile="map_checked.txt" #map file location
cell_json= "CELL_DEFINITIONS.json" #json file location conataining cell data
#some classes to look nice
class Hex:
    def __init__(self, x=0, y=0,name="",claim =0, owner = ""):
        self.x = x
        self.y = y
        self.name = name
        self.claim = claim
        self.owner = owner
    
class Hexmap:
    def __init__(self,x=1000,y=1000):
        self.hexarray = [[Hex()] * x for i in range(y)]
        self.xsize=x
        self.ysize=y
    
    def get(self, x, y):
        return self.hexarray[int(x+self.xsize/2)][int(y+self.ysize/2)]
    def set(self, x, y, sethex):
        self.hexarray[int(x+self.xsize/2)][int(y+self.ysize/2)] = sethex



Here we load the cell definitions from the json, and the map from the txt.  
The map is stored in the "hmap" variable.

In [2]:
#load json
file = open(cell_json,'r')
#because magic
test = file.read()
file.close()
test = test[test.find('['):]
cellinfo = json.loads(test)
#magic end

#load map
hmap = Hexmap()
with open(mapfile) as file:
    for line in file: 
        line = line.strip()
        parsetuple= tuple(line.split(','))
        try:
            parseclaim = str(int(parsetuple[3]))
        except ValueError:
            parseclaim = 0
        #hmap.set(int(parsetuple[0]),int(parsetuple[1]),Hex(int(parsetuple[0]),int(parsetuple[1]),parsetuple[2],int(parsetuple[3]),parsetuple[4]))
        parsedhex = Hex(int(parsetuple[0]), int(parsetuple[1]), parsetuple[2], parseclaim, parsetuple[4])
        hmap.set(int(parsetuple[0]),int(parsetuple[1]),parsedhex)

Defining function to use later.  
**traverse_hexes** is needed to get list of hexes near a point in a given radius. It is used later in other functions.  
**eval_MC** is an example evaluation function,which evaluates Mining Colony spots. (planet resources are counted once, field ones are halved.)  
Feel free to define you own evaluatinon method here.

In [3]:
#define

#aux function to get hexes near a point in a given radius
def traverse_hexes(x = 0, y = 0, radius = 500):
    traverse_list = []
    for ix in range(-radius,radius+1):
        if ix >=0:
            for iy in range(-radius,radius-ix+1):
                traverse_list.append((ix+x,iy+y))
        else:
            for iy in range(-radius-ix,radius+1):
                traverse_list.append((ix+x,iy+y))
    return traverse_list

#example evauation function. This evaluates Mining colony spots
def eval_MC(x,y, radius = 1, maxclaim = 200):
    hexlist=traverse_hexes(x,y,radius)
    value = 0
    #check if the spot is free
    if hmap.get(x,y).name != "":
        return value
    #iterate over surroundings
    for h in hexlist:
        if int(hmap.get(h[0],h[1]).claim) <= maxclaim: #check if its claimable
            match = [x for x in cellinfo if x["Name"] == hmap.get(h[0],h[1]).name] #match name
            if len(match) > 0:
                if match[0]["Type"] == 1: #its a planet
                    value = value + match[0]["HarvestValue"]["CR"] + match[0]["HarvestValue"]["GR"] + match[0]["HarvestValue"]["MR"]
                if match[0]["Type"] == 2: #its a field
                    value = value + 0.5 * (match[0]["HarvestValue"]["CR"] + match[0]["HarvestValue"]["GR"] + match[0]["HarvestValue"]["MR"])
    return value

Code for searching in a given area. The area is given by a point and a radius around. Keep in mind that hexes that values for hexes that aren't scanned are defaulted to 0. Currently only a radius of 50 is scanned around 70, -234.

In [6]:
#sample search
search_middle = 70, -234
search_radius = 50
spot_radius = 3
maxclaim = 300
value_threshold = 20 * 24
eval_function = eval_MC #set evaluation function

hlist = traverse_hexes(search_middle[0],search_middle[1],search_radius) #list of hexes to check
result = []

#evaluation
for h in hlist:
    value = eval_function(h[0],h[1],spot_radius,maxclaim)
    if value >= value_threshold:
        result.append((h,value))
print("eval done")

The results are stored in a list named **result** Here we sort it then print, so it can be viewed in a more pleasant format.

In [7]:
#sort
result.sort(key=lambda tup: tup[1], reverse=True)
#print nicely
for res in result:
    print("Value: "+str(res[1]/24)+" at: /goto "+ str(res[0][0])+' '+ str(res[0][1]))

Value: 30.0 at: /goto 25 -197
Value: 29.5 at: /goto 24 -196
Value: 29.0 at: /goto 25 -188
Value: 28.5 at: /goto 26 -221
Value: 28.0 at: /goto 43 -222
Value: 28.0 at: /goto 55 -227
Value: 27.5 at: /goto 25 -190
Value: 27.5 at: /goto 28 -226
Value: 27.5 at: /goto 33 -207
Value: 27.0 at: /goto 23 -218
Value: 27.0 at: /goto 26 -226
Value: 27.0 at: /goto 27 -227
Value: 27.0 at: /goto 27 -226
Value: 27.0 at: /goto 28 -225
Value: 27.0 at: /goto 52 -221
Value: 27.0 at: /goto 54 -225
Value: 26.5 at: /goto 23 -217
Value: 26.5 at: /goto 28 -211
Value: 26.5 at: /goto 30 -227
Value: 26.5 at: /goto 59 -265
Value: 26.0 at: /goto 24 -198
Value: 26.0 at: /goto 27 -225
Value: 26.0 at: /goto 30 -203
Value: 26.0 at: /goto 31 -229
Value: 26.0 at: /goto 36 -238
Value: 26.0 at: /goto 49 -224
Value: 26.0 at: /goto 49 -223
Value: 26.0 at: /goto 52 -253
Value: 26.0 at: /goto 57 -239
Value: 26.0 at: /goto 60 -264
Value: 26.0 at: /goto 100 -276
Value: 25.5 at: /goto 25 -189
Value: 25.5 at: /goto 27 -211
Value: 25